#### GISC 420 T1 2022
# Overlay and spatial join

In [ ]:
# You need to run this cell to get things setup
%matplotlib inline

import matplotlib
import matplotlib.pyplot as pyplot

import geopandas

The classic GIS operation *par excellence* is overlay of various kinds. Overlays are applied between two data layers, and are `GeoDataFrame` operations. In `geopandas`this means that the attribute data tables also get involved.

## Load a couple of different datasets
This time we need a couple of datasets that overlap in various ways, so we can understand how the overlay operations work. I made a couple for this purpose, so load them:

In [ ]:
s1 = geopandas.read_file('squares1.geojson')
s2 = geopandas.read_file('squares2.geojson')

In [ ]:
fig = pyplot.figure(figsize=(8,8))
ax = fig.add_subplot(111)

s1.plot(ax=ax, facecolor='b', alpha=0.3)
for p, label in zip(s1.geometry, s1.A):
    ax.annotate(xy=(p.centroid.x-0.4, p.centroid.y-0.25), text=label)
s2.plot(ax=ax, facecolor='r', alpha=0.3)

## `overlay` operations
OK. So let's see what happens when we use the `geopandas.overlay` function on these datasets.

In [ ]:
s1_o_s2 = geopandas.overlay(s1, s2, how="intersection")

fig = pyplot.figure(figsize=(8,8))
ax = fig.add_subplot(111)
s1.plot(ax=ax, facecolor='None', edgecolor='b')
s2.plot(ax=ax, facecolor='None', edgecolor='r')
s1_o_s2.plot(ax=ax, facecolor='m', linewidth=0, alpha=0.5)
for p, label in zip(s1_o_s2.geometry, s1_o_s2.A):
    ax.annotate(xy=(p.centroid.x-0.4, p.centroid.y-0.25), text=label)

So spatially, this has produced a `GeoDataFrame` that is the intersection of the two input layers. In attribute terms, we need to see what has happened also

In [ ]:
s1_o_s2

So elements in the new dataset inherit the attributes of both the input datasets.  The `how` option sent to the `overlay` function yields different results. The options are listed [here](https://geopandas.org/en/stable/docs/user_guide/set_operations.html). Give them each a try in the cell above and see what happens.  It's worth saying that not all of these will make sense in all situations.

It is also worth noting that `overlay` can currently only be applied between two polygon layers.

## `sjoin` operations
Overlay operations cause changes in the geometry of the included elements. Spatial join operations which are invoked by the `geopandas.sjoin()` function usually do not, but will append attributes from one dataset on to another depending on the spatial relation between the datasets.

In [ ]:
s1_j_s2 = geopandas.sjoin(s1, s2, how='inner', predicate='intersects')

fig = pyplot.figure(figsize=(8,8))
ax = fig.add_subplot(111)

s1.plot(ax=ax, facecolor='None', edgecolor='b', linewidth=1)
s2.plot(ax=ax, facecolor='None', edgecolor='r', linewidth=1)
s1_j_s2.plot(ax=ax, facecolor='m', linewidth=0, alpha=0.5)

for p, label in zip(s1_j_s2.geometry, s1_j_s2.A):
    ax.annotate(xy=(p.centroid.x-0.4, p.centroid.y-0.25), text=label)

In [ ]:
s1_j_s2

Here, the options available are that `how` can be `inner` (the default), `left`, or `right`. When set to `inner` only the cases in the first `GeoDataFrame` that satisfy the spatial constraint specified by the `predicate` setting are retained, and they acquire relevant attributes from both datasets.  When `how` is set to `left` or `right` all elements in the specified dataset are retained in the output, but there will not be data available to be joined unless the spatial constrain is met.

The spatial constraint specified by `predicate` can be any of `intersects`, `within` or `contains`.  Again, try experimenting with these options above to see what happens.

Spatial join operations work between different types of geometries in the ways you might expect.

By combining `sjoin`, `overlay` and operations such as `buffer` a very wide variety of spatial relationships between datasets can be implemented, although it sometimes may require several steps to accomplish a particular desired task. 